# Transformers: Walking Through the Math Behind Self Attention

### Briefly:<br>
**Transformers map input to output sequences**<br>
**Self attention assumes these sequences to have the same length**<br>
**Captures relevance of an input token for an output token**<br>
- Input vectors $x_1, x_2, \ldots, x_t, \in \mathbb{R}^{d}$
- Output vectors $y_1, y_2, \ldots, y_t, \in \mathbb{R}^{d}$
**Calculate output as a weighted sum**<br>
- - - 
### Barebones (walkthrough w/o query, key, value and corresponding matrices)
**General Equations:**<br>
$$\begin{aligned}
w'_{ij}=\frac{x'_{i}x_{j}}{\sqrt{d}}\\\\
w_{ij}=softmax(w'_{ij})=\frac{exp(w'_{ij})}{\sum_{j}exp(w'_{ij})}\\\\
y_i = \sum_{j=1}^{t}w_{ij}x_{j},\\
\end{aligned}$$
**Specific Case - Compute $y_{2}$:**<br>
Let $t=4$ and $i=2$, meaning we have four input (hence output) vectors and we are calculating output for the second one. 
*Note that here $i$ denotes the index of the vector which we are computing and  $j$ denotes the set over which we sum. So, $i$ is fixed and $j$ varies from $1$ to $4$.*

**Step 1)**<br>
Obtain weight matrices $w'_{ij}$: $w'_{21}, \quad w'_{22}, \quad w'_{23}, \quad w'_{24}$
$$
\begin{aligned}
w'_{21}=\frac{x'_{2}x_{1}}{\sqrt{d}}\\\\
w'_{22}=\frac{x'_{2}x_{2}}{\sqrt{d}}\\\\
w'_{23}=\frac{x'_{2}x_{3}}{\sqrt{d}}\\\\
w'_{24}=\frac{x'_{2}x_{4}}{\sqrt{d}}
\end{aligned}
$$
**Step 2)**<br>
Normalize the weight matrices using $softmax$
$$\begin{aligned}
w_{21}=\frac{exp(w'_{21})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\\\
w_{22}=\frac{exp(w'_{22})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\\\
w_{23}=\frac{exp(w'_{23})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\\\
w_{24}=\frac{exp(w'_{24})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\\\
\end{aligned}$$
**Step 3)**<br>
Calculate weighted sum
$$y_2=w_{21}x_{1}+w_{22}x_{2}+w_{23}x_{3}+w_{24}x_{4}$$
- - -
### Enter Query, Key, Value
Every input vector $i$ appears in three roles:
- Calculate weights for its own output $w_{ij}$ -- *query*
- Calculate weights for other output's weights $w_{zj}$, where $j=i$ -- *key*
- Part of the weighted sum to calculate each $y_i$ -- *value*

In the case of $x_2$, the above section showed where it played a role in computing weight matrices $w_{2j}$ and output $y_2$. If we were to demonstrate equations for computing $y_{1}, y_{3}, y_{4}$, input vector $x_i$ would play a role in computing $w_{12}, w_{32}, w_{42}$ in the three different computations separately, as well as in the relevant weighted sums. 

**Query:**
- $x_{i}$ appears in calculating weights for its own output $y_i$
- $x_2$ appears in calculating weights for its own output $y_{2}$

**Key:**
- $x_i$ appears in calculating weights for all outputs
- $x_2$ appears in calculating weights for outputs $y_1, y_2, y_3, y_4$

**Value:**
- $x_i$ appears in the weighted sum for all outputs
- $x_2$ appears in the weighted sum for $y_1, y_2, y_3, y_4$

**$W_{q}, W_{k}, W_{v}$**<br>
In practice we have additional trainable weight matrices $W_{q}, W_{k} \quad \text{and} \quad W_{v}$ each for query, key and value. These are $d \times d$ matrices and can be multiplied by our $d \times 1$ input vector $x_{i}$. 

Hence our $d \times 1$ vector $x_{i}$ gets transformed into three separate $d \times 1$ vectors:
$$
\begin{aligned}
W_{q}x_{i} = q_{i}\\\\
W_{k}x_{i} = k_{i}\\\\
W_{v}x_{i} = v_{i},
\end{aligned}
$$
which are used to compute the weight matrices $w'_{ij}$.

Note that in the above section $w'_{ij}=x'_{i}x_{j}$, where $x_{i}$ played the role of query - *i.e.* helped compute weights for its own output and $x_{j}$ played the role of key - *i.e.* helped compute weights for other input's output. 

Now using the relevant query and key vectors, we compute:
$$
\begin{aligned}
w'_{ij} = \frac{(W_{q}x_{i})'W_{k}x_{j}}{\sqrt{d}}\\\\
= \frac{q'_ik_j}{\sqrt{d}}
\end{aligned}
$$
where $w'_{ij}$ will be $1 \times 1$ scalar, as we multiply $(d \times 1)' (d \times 1) = (1 \times d)(d \times 1)$.

From here on, weight matrix $w_{ij}$ is computed as before:
$$
\begin{aligned}
w_{ij} = softmax(w'_{ij}),
\end{aligned}
$$
but again the final output computation changes from its previous form of $y_i = \sum_{j=1}^{t}w_{ij}x_{j}$ to
$$
\begin{aligned}
y_{i} = \sum_{j=1}^{t}w_{ij}W_{v}x_{j}\\\\
= \sum_{j=1}^{t}w_{ij}v_{j},
\end{aligned}
$$
where $y_i$ will also be of dimension $d \times 1$ as $w_{ij}$ is a scalar and $v_{j}$ is of dimension $d_{1}.$

**Back to Specific Case - Compute $y_{2}$:**<br>
**Step 1)**<br>
Obtain query vector $q_2$ using $x_2$
$$
\begin{aligned}
q_{2}=W_{q}x_2,
\end{aligned}
$$
Transpose of $q_2$ will play the role of query in this case, instead of $x'_2$ used previously.

**Step 2)**<br>
Obtain key vectors $k_1, k_2, k_3, k_4$ using $x_1, x_2, x_3, x_4$
$$
\begin{aligned}
k_1 = W_kx_1\\\\
k_2=W_kx_2\\\\
k_3=W_kx_3\\\\
k_4=W_kx_4
\end{aligned}
$$
**Step 3)**<br>
Compute matrices $w'_{ij}: w'_{21}, \quad w'_{22}, \quad w'_{23}, \quad w'_{24}$<br>
*Note that these matrices are exactly what we call attention scores.*
$$
\begin{aligned}
w'_{21} = \frac{q'_2k_1}{\sqrt{d}}\\\\
w'_{22} = \frac{q'_2k_2}{\sqrt{d}}\\\\
w'_{23} = \frac{q'_2k_3}{\sqrt{d}}\\\\
w'_{24} = \frac{q'_2k_4}{\sqrt{d}}
\end{aligned}
$$
**Step 4)**<br>
Normalize weight matrices (attention scores) using $softmax$
$$\begin{aligned}
w_{21}=\frac{exp(w'_{21})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\\\
w_{22}=\frac{exp(w'_{22})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\\\
w_{23}=\frac{exp(w'_{23})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\\\
w_{24}=\frac{exp(w'_{24})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\\\
\end{aligned}$$
**Step 5)**<br>
Obtain value vectors $v_1, v_2, v_3, v_4$ which will be used in the weighted sum to compute $y_2$ instead of previously used $x_1, x_2, x_3, x_4$:
$$
\begin{aligned}
v_1 = W_{v}x_1\\\\
v_2 = W_{v}x_2\\\\
v_3 = W_{v}x_3\\\\
v_4 = W_{v}x_4
\end{aligned}
$$
**Step 6)**<br>
Calculate weighted sum:
$$
\begin{align}
y_2 = w_{21}v_1 + w_{22}v_2 + w_{23}v_3 + w_{24}v_4
\end{align}
$$

In [3]:
# Required libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
from scipy.special import softmax
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

np.random.seed(66)
tf.random.set_seed(66)

### Demonstration of the $y_2$ Calculation in Python
Since in the above example have four vectors in total, we imagine we had a four-word-long sentence that we embedded as $d \times 1$ vectors $x_1, x_2, x_3, x_4$. To make things a bit more confusing, we can let $d=1$, meaning our embedding dimension is $4$ and $x_i$ is a vector of $4 \times 1$ dimension. 

**Step 0): Generate $x_1, x_2, x_3, x_4$**

In [12]:
# Define embedding dimension
d = 4
# Randomly initialize embedded vectors
x_1 = tf.random.normal((d,1))
x_2 = tf.random.normal((d,1))
x_3 = tf.random.normal((d,1))
x_4 = tf.random.normal((d,1))

print(f'Our vectors are:\n'
      f'x_1:\n{x_1}\n'
      f'x_2:\n{x_2}\n'
      f'x_3:\n{x_3}\n'
      f'x_4:\n{x_4}\n'
      f'With shapes: {x_1.shape}, {x_2.shape}, {x_3.shape}, {x_4.shape}')

Our vectors are:
x_1:
[[-0.5901138 ]
 [ 0.10746985]
 [-0.68417746]
 [-1.087325  ]]
x_2:
[[ 0.06799816]
 [ 0.26192757]
 [ 2.078989  ]
 [-0.15885977]]
x_3:
[[-0.4177143 ]
 [-0.6695858 ]
 [-0.05910626]
 [ 1.4612687 ]]
x_4:
[[-0.27432463]
 [-0.44688827]
 [-0.7900979 ]
 [-0.03789218]]
With shapes: (4, 1), (4, 1), (4, 1), (4, 1)


---

**Step 0): Generate $W_q, W_k, W_v$**

In [13]:
W_q = tf.random.normal((d,d))
W_k = tf.random.normal((d,d))
W_v = tf.random.normal((d,d))

print(f'Our matrices are:\n'
      f'W_q:\n{W_q}\n'
      f'W_k:\n{W_k}\n'
      f'W_v:\n{W_v}\n'
      f'With shapes: {W_q.shape}, {W_k.shape}, {W_v.shape}')

Our matrices are:
W_q:
[[-1.3717711  -0.98146987  0.8260041  -1.2923543 ]
 [ 0.30707964  0.786002   -2.103323   -1.7300906 ]
 [-0.5035843  -0.4742107  -0.2712986  -0.63562334]
 [ 1.2032156   0.18251911 -1.4129548   1.0945445 ]]
W_k:
[[-1.6024884  -1.1137156  -0.04244602  0.34064332]
 [-1.2472221   0.23090297 -0.01369113  0.97045213]
 [ 0.4897503   0.4532431  -0.56273043  0.02543709]
 [-1.9602743  -1.1043301  -0.9142158   0.13891058]]
W_v:
[[ 4.4455358e-01 -7.1996415e-01  9.7427595e-01 -7.3896867e-01]
 [ 2.7865940e-01 -1.2198892e+00 -4.6462238e-01  1.2124052e+00]
 [ 1.2106689e+00  1.3768704e+00 -2.1070539e-04  4.4651002e-01]
 [ 9.5693940e-01  1.3449348e+00  1.0805776e+00 -5.2276546e-01]]
With shapes: (4, 4), (4, 4), (4, 4)


---

**Step 1): Obtain query vector $q_2$ using $x_2$**
$$
\begin{aligned}
q_{2}=W_{q}x_2,
\end{aligned}
$$

In [14]:
# Multiply 4x4 W_q with 4x1 x_2
q_2 = tf.matmul(W_q, x_2)

print(f'q_2:\n{q_2}\n'
      f'With shape: {q_2.shape}')

q_2:
[[ 1.5722046]
 [-3.8711874]
 [-0.6215035]
 [-2.9817736]]
With shape: (4, 1)


---

**Step 2) Obtain key vectors $k_1, k_2, k_3, k_4$ using $x_1, x_2, x_3, x_4$**
$$
\begin{aligned}
k_1 = W_kx_1\\
k_2=W_kx_2\\
k_3=W_kx_3\\
k_4=W_kx_4
\end{aligned}
$$

In [15]:
k_1 = tf.matmul(W_k, x_1)
k_2 = tf.matmul(W_k, x_2)
k_3 = tf.matmul(W_k, x_3)
k_4 = tf.matmul(W_k, x_4)

print(f'Our key vectors are:\n'
      f'k_1:\n{k_1}\n'
      f'k_2:\n{k_2}\n'
      f'k_3:\n{k_3}\n'
      f'k_4:\n{k_4}\n'
      f'With shapes: {k_1.shape}, {k_2.shape}, {k_3.shape}, {k_4.shape}')

Our key vectors are:
k_1:
[[ 0.48461038]
 [-0.28501165]
 [ 0.11705065]
 [ 1.5125477 ]]
k_2:
[[-0.5430384 ]
 [-0.20695846]
 [-1.0219324 ]
 [-2.3452613 ]]
k_3:
[[ 1.9153907 ]
 [ 1.7852738 ]
 [-0.43762955]
 [ 1.8153    ]]
k_4:
[[0.95793724]
 [0.21300066]
 [0.10674866]
 [1.7483201 ]]
With shapes: (4, 1), (4, 1), (4, 1), (4, 1)


---

**Step 3) Compute attention scores $w'_{ij}: w'_{21}, \quad w'_{22}, \quad w'_{23}, \quad w'_{24}$**
$$
\begin{aligned}
w'_{21} = \frac{q'_2k_1}{\sqrt{d}}\\\\
w'_{22} = \frac{q'_2k_2}{\sqrt{d}}\\\\
w'_{23} = \frac{q'_2k_3}{\sqrt{d}}\\\\
w'_{24} = \frac{q'_2k_4}{\sqrt{d}}
\end{aligned}
$$

In [16]:
# Define scaling factor - square root of d 
scale_factor = tf.math.sqrt(float(d))

# Transpose q_2
q_T_2 = tf.transpose(q_2)

# Obtain attention scores
w_T_21 = tf.matmul(q_T_2, k_1) / scale_factor
w_T_22 = tf.matmul(q_T_2, k_2) / scale_factor
w_T_23 = tf.matmul(q_T_2, k_3) / scale_factor
w_T_24 = tf.matmul(q_T_2, k_4) / scale_factor

print(f'Our attention scores are:\n'
      f'w_T_21:\n{w_T_21}\n'
      f'w_T_22:\n{w_T_22}\n'
      f'w_T_23:\n{w_T_23}\n'
      f'w_T_24:\n{w_T_24}\n'
      f'With shapes: {w_T_21.shape}, {w_T_22.shape}, {w_T_23.shape}, {w_T_24.shape}')

Our attention scores are:
w_T_21:
[[-1.3587911]]
w_T_22:
[[3.78779]]
w_T_23:
[[-4.5202847]]
w_T_24:
[[-2.2989657]]
With shapes: (1, 1), (1, 1), (1, 1), (1, 1)


---

**Step 4) Normalize attention scores using $softmax$**
$$\begin{aligned}
w_{21}=\frac{exp(w'_{21})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\
w_{22}=\frac{exp(w'_{22})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\
w_{23}=\frac{exp(w'_{23})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\
w_{24}=\frac{exp(w'_{24})}{exp(w'_{21})+exp(w'_{22})+exp(w'_{23})+exp(w'_{24})}\\
\end{aligned}$$

In [17]:
# Take exponentials
exp_w_T_21 = tf.exp(w_T_21)
exp_w_T_22 = tf.exp(w_T_22)
exp_w_T_23 = tf.exp(w_T_23)
exp_w_T_24 = tf.exp(w_T_24)

# Calculate sum
exp_sum = exp_w_T_21 + exp_w_T_22 + exp_w_T_23 + exp_w_T_24

# Calculate normalized scores
w_21 = exp_w_T_21 / exp_sum
w_22 = exp_w_T_22 / exp_sum
w_23 = exp_w_T_23 / exp_sum
w_24 = exp_w_T_24 / exp_sum

print(f'Our normalized attention scores are:\n'
      f'w_21:\n{w_21}\n'
      f'w_22:\n{w_22}\n'
      f'w_23:\n{w_23}\n'
      f'w_24:\n{w_24}\n'
      f'With shapes: {w_21.shape}, {w_22.shape}, {w_23.shape}, {w_24.shape}')

Our normalized attention scores are:
w_21:
[[0.00577114]]
w_22:
[[0.99173045]]
w_23:
[[0.00024448]]
w_24:
[[0.00225398]]
With shapes: (1, 1), (1, 1), (1, 1), (1, 1)


---

**Step 5) Obtain value vectors $v_1, v_2, v_3, v_4$** 
$$
\begin{aligned}
v_1 = W_{v}x_1\\
v_2 = W_{v}x_2\\
v_3 = W_{v}x_3\\
v_4 + W_{v}x_4
\end{aligned}
$$

In [18]:
v_1 = tf.matmul(W_v, x_1)
v_2 = tf.matmul(W_v, x_2)
v_3 = tf.matmul(W_v, x_3)
v_4 = tf.matmul(W_v, x_4)

print(f'Our key vectors are:\n'
      f'v_1:\n{v_1}\n'
      f'v_2:\n{v_2}\n'
      f'v_3:\n{v_3}\n'
      f'v_4:\n{v_4}\n'
      f'With shapes: {v_1.shape}, {v_2.shape}, {v_3.shape}, {v_4.shape}')

Our key vectors are:
v_1:
[[-0.2027902 ]
 [-1.2959363 ]
 [-1.0518178 ]
 [-0.59105414]]
v_2:
[[ 1.9845519 ]
 [-1.4591215 ]
 [ 0.37159303]
 [ 2.746901  ]]
v_3:
[[-0.84103614]
 [ 2.4995322 ]
 [-0.775163  ]
 [-2.1280463 ]]
v_4:
[[-0.54198074]
 [ 0.7898675 ]
 [-0.9641763 ]
 [-1.697501  ]]
With shapes: (4, 1), (4, 1), (4, 1), (4, 1)


---

**Step 6) Calculate weighted sum $y_2$**
$$
\begin{align}
y_2 = w_{21}v_1 + w_{22}v_2 + w_{23}v_3 + w_{24}v_4
\end{align}
$$

In [19]:
# * operator since attention weights are scalars
y_2 = w_21 * v_1 + w_22 * v_2 + w_23 * v_3 + w_24 * v_4

print(f'Our output vector:\n'
      f'y_2:\n{y_2}\n'
      f'With shape: {y_2.shape}')

Our output vector:
y_2:
[[ 1.965543  ]
 [-1.4521428 ]
 [ 0.36008716]
 [ 2.716428  ]]
With shape: (4, 1)


---